## Install Requirments

In [2]:
! pip3 install -r ../../requirements.txt

Save vars

In [3]:
from dotenv import dotenv_values

env_name = "../../.env"
config = dotenv_values(env_name)

try:
    #Name that will be using to start all of the resources name
    base_name = config["BASE_NAME"]

    #OpenAI Endpoint and key
    ai_search_endpoint = config["AI_SEARCH_ENDPOINT"]
    ai_search_key = config["AI_SEARCH_KEY"]

    #Embedding Model details
    azure_openai_endpoint_embedding = config["AZURE_OPENAI_ENDPOINT_EMBEDDING"]
    azure_openai_key_embedding = config["AZURE_OPENAI_KEY_EMBEDDING"]
    embedding_model = config["AI_SEARCH_EMBEDDING_MODEL"]
except KeyError as e:
    print(f"Missing key: {e}")
    print(f"Please make sure you have a {env_name} file with all the required keys")


## Create vector database and correct fields

In [5]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchableField,
    SearchFieldDataType,
    SimpleField,
    SearchIndex,
    SearchFieldDataType,
    ComplexField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch
)
from azure.core.credentials import AzureKeyCredential
import logging

logging.info(f"index_SC_SDK_vector.py: {base_name}")
ai_search_credential = AzureKeyCredential(ai_search_key)


index_name = "{}-index-vector".format(base_name)

index_client = SearchIndexClient(
    endpoint=ai_search_endpoint, credential=ai_search_credential
    )


index = fields = [
        SimpleField(name="Id", type=SearchFieldDataType.String, key=True, filterable=True, sortable=True),
        
        SearchField(name="content", type=SearchFieldDataType.String, searchable=True, filterable=True, sortable=True),

        SearchField(name="Content_vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    retrievable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
    ]


vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind="hnsw",
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric="cosine"
            )
        )
    ],

    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
            vectorizer_name="myOpenai"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="myOpenai",
            kind = "azureOpenAI",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint_embedding,
                deployment_name=embedding_model,
                api_key=azure_openai_key_embedding,
                kind="azureOpenAI",
                model_name= embedding_model
            )
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="Id"),
        keywords_fields=[
            SemanticField(field_name="content"),
            ],
        content_fields=[
            SemanticField(field_name="content"),
            ]
    )
)

semantic_settings = SemanticSearch(configurations=[semantic_config])

index = SearchIndex(
    name= index_name,
    fields= fields,
    vector_search=vector_search,
    semantic_search=semantic_settings
    )



result = index_client.create_or_update_index(index)
print(f'Index - vector"{result.name}" created or updated successfully.')

Index - vector"jupyter-test-index-vector" created or updated successfully.
